# First visit Monte Carlo Policy Evaluation

┌─┬─┬─┬─┐

│0 │1 │2 │3 │

├─┼─┼─┼─┤

│4 │X │5 │6 │

├─┼─┼─┼─┤

│7 │8 │9 │10│

└─┴─┴─┴─┘

3 : 1점, 6 : -1점

0 왼 1 오 2 위 3 아래 

# V

In [2]:
import numpy as np
from copy import deepcopy

testmode = 0

# set parameters ###############################################################
epoch = 10000
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3]  
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

# policy
if 0: 
    # bad policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif 0: 
    # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif 0: 
    # optimal policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
elif 1: 
    # optimal policy + noise 
    # we use optimal policy with probability 1/(1+ep)
    # we use random policy with probability ep/(1+ep)
    ep = 0.1
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
    policy = policy + (ep/4)*np.ones((N_STATES, N_ACTIONS))
    policy = policy / np.sum(policy, axis=1).reshape((N_STATES,1))

# Every-Visit Monte-Carlo Policy Evaluation
# n_visits records number of visits for each state
# cum_gains records cumulative gains, i.e., sum of gains for each state
# where
# gain = reward + gamma * next_reward + gamma^2 * ...

n_visits = np.zeros(N_STATES)
cum_gains = np.zeros(N_STATES)

for _ in range(epoch):
    if testmode == 1 :
        print ("안녕")
        print (_,"번 째")
    # simulation_history records visited states including the terminal states 3 and 6
    # reward_history records occured rewards including final rewards 1. and -1. 
    simulation_history = []
    reward_history = []

    # indicate game is not over yet
    done = False
    
    # choose initial state randomly, not from 3 or 6
    s = np.random.choice([0, 1, 2, 4, 5, 7, 8, 9, 10])  
    NUM = 0 
    while not done:
        if testmode == 1 :
            print (NUM)
        # choose action using current policy
        a = np.random.choice(actions, p=policy[s, :])
        simulation_history.append(s)
        reward_history.append(R[s,a])
        if testmode == 1 :
            print ("si_hi : ", simulation_history)
            print ("re_hi : ", reward_history)
        # choose next state using transition probabilities
        s1 = np.random.choice(states, p=P[s, a, :])

        if s1 == 3:
            # if game is over, 
            # ready to break while loop by letting done = True
            # append end result to simulation_history 
            done = True
            simulation_history.append(s1)
            reward_history.append(R[s1,0])
        elif s1 == 6:
            # if game is over, 
            # ready to break while loop by letting done = True
            # append end result to simulation_history 
            done = True
            simulation_history.append(s1)
            reward_history.append(R[s1,0])
        else:
            # if game is not over, continue playing game
            s = s1
        
        NUM += 1

    # reward_history records occured rewards including final rewards 1 and -1  
    simulation_history = np.array(simulation_history)
    reward_history = np.array(reward_history)
    n = len(reward_history)
    
    # gain_history records occured gains 
    gain_history = deepcopy(reward_history) 
    
    if testmode == 1 :
        print ("gain_pre : ", gain_history)
    for i, reward in enumerate(reward_history[:-1][::-1]):
        gain_history[n-i-2] = reward + gamma * gain_history[n-i-2+1]
        if testmode == 1 :
            print ("i:",i,"reward",reward)
            print (gain_history)
    
    
    if testmode == 1 :
        print ("gain_post :", gain_history)
    
    check = [-1 for i in range(N_STATES)]
    
    for i in range(N_STATES):
        bbbbbb=-1
        for j in range(n) :
            if simulation_history[j] == i and bbbbbb == -1 :
                bbbbbb = j
        check [i] = bbbbbb
        
    if testmode == 1 :
        print ("check :", check)
        
    # update n_visits and cum_gains     
    for i in range(N_STATES):
        if check[i] != -1:
            n_visits[i] += 1
            cum_gains[i] += gain_history[check[i]]
        
    if testmode == 1 :
        print ("n_vi : ", n_visits)
        print ("cum_ga : ", cum_gains)
                        
V = cum_gains / (n_visits + 1.0e-8) 
print(V)

[ 0.83452835  0.87895072  0.9196893   1.          0.79492031  0.65281199
 -1.          0.75337609  0.71636092  0.67149823  0.41075439]


# Q

In [4]:
# test

testmode = 0

import numpy as np
from copy import deepcopy

# set parameters ###############################################################
epoch = 50000
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3]  
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

# policy
if 0: 
    # bad policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif 0: 
    # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif 0: 
    # optimal policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
elif 1: 
    # optimal policy + noise 
    # we use optimal policy with probability 1/(1+ep)
    # we use random policy with probability ep/(1+ep)
    ep = 0.1
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
    policy = policy + (ep/4)*np.ones((N_STATES, N_ACTIONS))
    policy = policy / np.sum(policy, axis=1).reshape((N_STATES,1))

# Every-Visit Monte-Carlo Policy Evaluation
# n_visits records number of visits for each state and action
# cum_gains records cumulative gains, i.e., sum of gains for each state and action
# where
# gain = reward + gamma * next_reward + gamma^2 * ...

# Previously for V
# n_visits = np.zeros(N_STATES)
# cum_gains = np.zeros(N_STATES)

print ("policy")
print (policy)

n_visits = np.zeros((N_STATES, N_ACTIONS))
cum_gains = np.zeros((N_STATES, N_ACTIONS))

for _ in range(epoch):
    if _ % 5000 == 0 :
        print (_,"/",epoch)
    if testmode == 1 :
        print ("안녕")
        print (_,"번 째")

    # simulation_history records visited states and actions including the terminal states 3 and 6
    # reward_history records occured rewards including final rewards 1 and -1 
    simulation_history = []
    reward_history = []

    # indicate game is not over yet
    done = False
    
    # choose initial state randomly, not from 3 or 6
    s = np.random.choice([0, 1, 2, 4, 5, 7, 8, 9, 10])  
    NUM = 0
    while not done:
        # choose action using current policy
        a = np.random.choice(actions, p=policy[s, :])
        
        # Previously for V 
        # simulation_history.append(s)
        
        simulation_history.append((s,a))
        reward_history.append(R[s,a])
        
        if testmode == 1 :
            print ("si_hi : ", simulation_history)
            print ("re_hi : ", reward_history)
            
        # choose next state using transition probabilities
        s1 = np.random.choice(states, p=P[s, a, :])

        if s1 == 3:
            # if game is over, 
            # ready to break while loop by letting done = True
            # append end result to simulation_history 
            done = True
            
            # Previously for V 
            # simulation_history.append(s1)
            
            simulation_history.append((s1,0))
            reward_history.append(R[s1,0])
        elif s1 == 6:
            # if game is over, 
            # ready to break while loop by letting done = True
            # append end result to simulation_history 
            done = True
            
            # Previously for V 
            # simulation_history.append(s1)
            
            simulation_history.append((s1,0))
            reward_history.append(R[s1,0])
        else:
            # if game is not over, continue playing game
            s = s1
        NUM += 1

    # reward_history records occured rewards including final rewards 1 and -1  
    reward_history = np.array(reward_history)
    n = len(reward_history)
    # gain_history records occured gains 
    gain_history = deepcopy(reward_history) 
    for i, reward in enumerate(reward_history[:-1][::-1]): 
        gain_history[n-i-2] = reward + gamma * gain_history[n-i-2+1]
        
    # update n_visits and cum_gains 
    # Previously for V 
    # for i in range(N_STATES):
    #     n_visits[i] += np.sum(simulation_history==i) 
    #     cum_gains[i] += np.sum(gain_history[simulation_history==i]) 
    
    if testmode == 1 :
        print ("simu_hi : ", simulation_history)
        print ("gain_hi : ", gain_history)
    
    check = [-1 for i in range(N_STATES)]
    
    for i in range(N_STATES):
        bbbbbb=-1
        for j in range(n) :
            if simulation_history[j][0] == i and bbbbbb ==-1 :
                bbbbbb = j
        check [i] = bbbbbb
        
    if testmode == 1 :
        print ("check :", check)
    
    for i, (s, a) in enumerate(simulation_history): 
        if check[s]==i :
            n_visits[s, a] += 1.
            cum_gains[s, a] += gain_history[i]
            
    if testmode == 1 :
        print ("n_vi : ", n_visits)
        print ("cum_ga : ", cum_gains)

Q = cum_gains / (n_visits + 1.0e-8) 
print("Q", Q)

policy
[[0.02272727 0.93181818 0.02272727 0.02272727]
 [0.02272727 0.93181818 0.02272727 0.02272727]
 [0.02272727 0.93181818 0.02272727 0.02272727]
 [0.02272727 0.93181818 0.02272727 0.02272727]
 [0.02272727 0.02272727 0.93181818 0.02272727]
 [0.02272727 0.02272727 0.93181818 0.02272727]
 [0.02272727 0.02272727 0.93181818 0.02272727]
 [0.02272727 0.02272727 0.93181818 0.02272727]
 [0.93181818 0.02272727 0.02272727 0.02272727]
 [0.93181818 0.02272727 0.02272727 0.02272727]
 [0.93181818 0.02272727 0.02272727 0.02272727]]
0 / 50000
5000 / 50000
10000 / 50000
15000 / 50000
20000 / 50000
25000 / 50000
30000 / 50000
35000 / 50000
40000 / 50000
45000 / 50000
Q [[ 0.8124048   0.83628846  0.81700873  0.78564795]
 [ 0.82047735  0.88157714  0.86164418  0.84349861]
 [ 0.8303344   0.92612563  0.89593137  0.70326968]
 [ 1.          0.          0.          0.        ]
 [ 0.75992865  0.76643904  0.79851278  0.73531867]
 [ 0.68300958 -0.65478403  0.66757181  0.51155294]
 [-1.          0.          0.   